<a href="https://colab.research.google.com/github/shahnazumer/LECL-LANGCHAIN/blob/main/LangChain_Expression_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken
!pip -q install chromadb

In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

# LangChain Expression Language


In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.schema.output_parser import StrOutputParser

In [5]:
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
    )


In [76]:
prompt = ChatPromptTemplate.from_template(
    "tell me an intersting fact about {subject}"
    )

In [77]:
chain = prompt | model

In [78]:
chain.invoke({"subject": "Elvis"})

AIMessage(content='One interesting fact about Elvis Presley is that he was a black belt in karate. He began studying martial arts in the 1950s and eventually earned his black belt in 1960. Elvis was passionate about karate and even incorporated some of the moves into his performances. He often practiced with his friends and bodyguards, and even had a custom-made karate uniform with his name embroidered on it.')

In [79]:
chain = prompt | model | StrOutputParser()

In [80]:
chain.invoke({"subject": "Elvis"})

'One interesting fact about Elvis Presley is that he was a black belt in karate. He began studying martial arts in the 1950s and eventually earned his black belt in 1960. Elvis was passionate about karate and even incorporated some of the moves into his performances. He often practiced with his friends and bodyguards, and even had a custom-made karate uniform with his name embroidered on it.'

In [85]:
# Streaming Example
for chunk in chain.stream({"subject": "Elvis"}):
    print(chunk, end="", flush=True)

One interesting fact about Elvis Presley is that he was a black belt in karate. He began studying martial arts in the 1950s and eventually earned his black belt in 1960. Elvis was passionate about karate and even incorporated some of the moves into his performances. He often practiced with his friends and bodyguards, and even had a custom-made karate uniform with his name embroidered on it.

In [87]:
# Batch Processing Example
subject_batch = [
    {"subject": "Elvis"},
    # Add more notes here
]
batch_view = chain.batch(subject_batch)

## Bindings

In [12]:
prompt = ChatPromptTemplate.from_template(
    "tell me 3 intersting facts about {subject}"
    )

In [13]:
chain = prompt | model.bind(stop=["\n"]) | StrOutputParser()


In [14]:
chain.invoke({"subject": "Elvis"})

'1. Elvis Presley, often referred to as the "King of Rock and Roll," was born on January 8, 1935, in Tupelo, Mississippi. He began his music career in the mid-1950s and quickly rose to fame with his unique blend of rockabilly, country, and rhythm and blues.'

## Adding OpenAI Functions

In [15]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]
functions_chain = prompt | model.bind(function_call= {"name": "joke"}, functions= functions)

In [16]:
functions_chain.invoke({"subject": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "setup": "Why don\'t bears wear shoes?",\n  "punchline": "Because they have bear feet!"\n}', 'name': 'joke'}})

### Functions Output Parser


In [17]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

functions_chain = (
    prompt
    | model.bind(function_call= {"name": "joke"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [18]:
response = functions_chain.invoke({"subject": "bears"})

response

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

In [19]:
response['punchline']

'Because they have bear feet!'

In [20]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

functions_chain = (
    prompt
    | model.bind(function_call= {"name": "joke"}, functions= functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [21]:
functions_chain.invoke({"subject": "bears"})

"Why don't bears wear shoes?"

## LECL Case: Retrieval Augmented Generation (RAG)

In [64]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

In [65]:
# Create the retriever
fake_docs = ["James bond works for MI6","Bond is a spy",
             "James Bond has a licence to kill", "James Bond likes cats"]
vectorstore = Chroma.from_texts(fake_docs, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


In [66]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [67]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [68]:
chain.invoke("Who is James Bond?")

'James Bond is a person who works for MI6.'

In [69]:
chain.invoke("What does James Bond like to do?")

'Based on the given context, it is not possible to determine what James Bond likes to do.'

In [70]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language")
} | prompt | model | StrOutputParser()

In [71]:
chain.invoke({"question": "where does James work?", "language": "english"})

'James works for MI6.'

In [73]:
chain.invoke({"question": "where does James work?", "language": "russain"})

'Джеймс работает в МИ6.'